# Feature correlation analysis

hello

In [1]:
import pandas as pd 
import os
from pathlib import Path

import numpy as np

In [2]:
pwd  = Path(os.getcwd())
pwd

PosixPath('/Users/koray/data-literacy-group13/data')

In [3]:
dataset_csv = "ICLR.cc-2023-Conference.csv"

csv_path = pwd / dataset_csv
df = pd.read_csv(csv_path)

df

,title,keywords,TL;DR,abstract,cdate,tmdate,reviewer1_recommendation,reviewer1_confidence,reviewer1_summary_of_the_paper,reviewer1_strength_and_weaknesses,...,reviewer4_clarity_quality_novelty_and_reproducibility,reviewer4_summary_of_the_review,reviewer4_correctness,reviewer4_technical_novelty_and_significance,reviewer4_empirical_novelty_and_significance,reviewer4_flag_for_ethics_review,reviewer4_length,reviewer4_cdate,reviewer4_tmdate,decision
0,Guiding Safe Exploration with Weakest Precondi...,reinforcement learning; safe learning; safe ex...,"We use an online, weakest-precondition-based a...",In reinforcement learning for safety-critical ...,2022-09-22 14:36:24,2024-11-25 10:13:40,6: marginally above the acceptance threshold,"4: You are confident in your assessment, but n...",This paper deals with safe exploration in rein...,### Strength\n- Interesting and importance pro...,...,The paper is generally clear. I appreciate the...,I like the proposed approach and its associate...,4: All of the claims and statements are well-s...,3: The contributions are significant and somew...,2: The contributions are only marginally signi...,NO.,3082.0,2022-11-03 08:08:39,2022-12-14 03:46:36,Accept: poster
1,An Adaptive Entropy-Regularization Framework f...,Multi-Agent Reinforcement Learning; Entropy Re...,This paper proposes an adaptive entropy-regul...,"In this paper, we propose an adaptive entropy-...",2022-09-22 14:33:17,2023-02-14 00:28:54,"3: reject, not good enough","4: You are confident in your assessment, but n...",This paper presents an MARL algorithm to adpat...,### Strengths\n\n1. This paper studies adaptiv...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Reject
2,AutoSparse: Towards Automated Sparse Training,sparsity; sparse training; deep learning,NaN,Sparse training is emerging as a promising ave...,2022-09-22 14:32:39,2023-02-14 00:29:06,5: marginally below the acceptance threshold,"4: You are confident in your assessment, but n...",This paper first introduces a technique called...,Strengths:\n\n- Provided intuitive examples an...,...,See the strength and weaknesses.,1. The proposed method seems tricky and not el...,2: Several of the paper’s claims are incorrect...,2: The contributions are only marginally signi...,1: The contributions are neither significant n...,NO.,2254.0,2022-10-27 20:14:54,2022-10-27 20:14:54,Reject
3,Sampling is as easy as learning the score: the...,diffusion models; score-based generative model...,We prove that given an L2-accurate score estim...,We provide theoretical convergence guarantees ...,2022-09-22 14:40:10,2023-02-14 00:27:00,"8: accept, good paper","4: You are confident in your assessment, but n...",This paper examines the convergence of SGMs un...,Strength:\n\n- The presentation is clear.\n\n-...,...,"The paper is very clear, despite the complex s...",Overall I think the paper introduces very stro...,3: Some of the paper’s claims have minor issue...,3: The contributions are significant and somew...,Not applicable,NO.,5316.0,2022-10-27 22:09:32,2022-10-27 22:09:32,Accept: notable-top-5%
4,RoCourseNet: Distributionally Robust Training ...,Counterfactual Explanation; Algorithmic Recour...,NaN,Counterfactual (CF) explanations for machine l...,2022-09-22 14:38:43,2024-11-25 10:12:40,"3: reject, not good enough","4: You are confident in your assessment, but n...",This submission proposed an approach to solvin...,Strength:\n\n1. The paper is easy to follow.\n...,...,The paper is written clearly and is easy to fo...,The paper solves an important problem. The exp...,4: All of the claims and statements are well-s...,2: The contributions are only marginally signi...,2: The contributions are only marginally signi...,NO.,2937.0,2022-10-29 20:23:57,2022-10-29 20:23:57,Reject
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,Towards Understanding Robust Memorization in A...,adversarial robustness; adversarial training; ...,We provide a theoretical understanding of adve...

In [4]:
# set correct data types

date_cols = ['cdate', 'tmdate'] \
            + [f"reviewer{i}_cdate" for i in range(1, 5)] \
            + [f"reviewer{i}_tmdate" for i in range(1, 5)]
for col_name in date_cols:
    df[col_name] = pd.to_datetime(df[col_name])


# -----------------------------------------------------------------------------

# set title to str
df['title'] = df['title'].astype(str)

# -----------------------------------------------------------------------------

# split keywords into a list

df['keywords'] = df['keywords'].apply(lambda x: [] if pd.isna(x) else x.split(';'))




In [5]:
df.dtypes

title                                                            object
keywords                                                         object
TL;DR                                                            object
abstract                                                         object
cdate                                                    datetime64[ns]
tmdate                                                   datetime64[ns]
reviewer1_recommendation                                         object
reviewer1_confidence                                             object
reviewer1_summary_of_the_paper                                   object
reviewer1_strength_and_weaknesses                                object
reviewer1_clarity_quality_novelty_and_reproducibility            object
reviewer1_summary_of_the_review                                  object
reviewer1_correctness                                            object
reviewer1_technical_novelty_and_significance                    

In [6]:
# build a feature df
df_features = df.iloc[:, [0,1,4,5]].copy()
df_features

,title,keywords,cdate,tmdate
0,Guiding Safe Exploration with Weakest Precondi...,"[reinforcement learning, safe learning, safe...",2022-09-22 14:36:24,2024-11-25 10:13:40
1,An Adaptive Entropy-Regularization Framework f...,"[Multi-Agent Reinforcement Learning, Entropy ...",2022-09-22 14:33:17,2023-02-14 00:28:54
2,AutoSparse: Towards Automated Sparse Training,"[sparsity, sparse training, deep learning]",2022-09-22 14:32:39,2023-02-14 00:29:06
3,Sampling is as easy as learning the score: the...,"[diffusion models, score-based generative mod...",2022-09-22 14:40:10,2023-02-14 00:27:00
4,RoCourseNet: Distributionally Robust Training ...,"[Counterfactual Explanation, Algorithmic Reco...",2022-09-22 14:38:43,2024-11-25 10:12:40
...,...,...,...,...
3791,Towards Understanding Robust Memorization in A...,"[adversarial robustness, adversarial training...",2022-09-22 14:35:25,2023-02-14 00:28:19
3792,Identifying Phase Transition Thresholds of Per...,[],2022-09-22 14:42:21,2023-02-14 00:26:26
3793,BEVDistill: Cross-Modal BEV Distillation for M...,"[object detection, 3d detection, BEV percept...",2022-09-22 14:30:15,2024-11-25 10:16:48
3794,Rethinking Deep Spiking Neural Networks: A Mul...,"[spiking neural network, multi-layer perceptr...",2022-09-22 14:35:28,2023-02-14 00:28:18


In [7]:
# calculate features

import re

# -----------------------------------------------------------------------------

df_features['decision_boolean'] = df['decision'].apply(lambda x: 'accept' in x.lower())

# -----------------------------------------------------------------------------


def _is_ethics_review_flagged(maybe_string) -> bool:
    # na check
    if isinstance(maybe_string, float) and np.isnan(maybe_string):
        return np.nan
    return 'yes' in maybe_string.lower()


ethics_review_flags = [f"reviewer{i}_flag_for_ethics_review" for i in range(1, 5)]
for reviewer_flag in ethics_review_flags:
    df_features[reviewer_flag] = df[reviewer_flag].apply(_is_ethics_review_flagged)


# -----------------------------------------------------------------------------

# reviwer scores

def _extract_numeric_prefix(maybe_string: object) -> int:
    """
    Extracts the leading integer from a string.
    If no integer is found, returns NaN.
    """
    string = str(maybe_string)
    match = re.match(r'^(\d+)', string.strip())
    return int(match.group(1)) if match else np.nan


reviewer_number_cols = [col for col in df.columns if col.startswith('reviewer')
                        and col.endswith((
                            'recommendation',
                            'confidence',
                            'correctness',
                            'technical_novelty_and_significance',
                            'empirical_novelty_and_significance',
                        ))]

for col in reviewer_number_cols:
    df_features[col] = df[col].apply(_extract_numeric_prefix)


# -----------------------------------------------------------------------------

reviewer_date_cols = [col for col in df.columns if col.startswith('reviewer')
                        and col.endswith((
                            'cdate',
                            'tmdate',
                        ))]

for col in reviewer_date_cols:
    df_features[col] = df[col]


# -----------------------------------------------------------------------------

# reviwer effort in rewviewing the paper
for reviewer in [f"reviewer{i}" for i in range(1, 5)]:
    reviewer_cols = [col for col in df.columns if col.startswith(reviewer)]
    df_features[f"{reviewer}_length"] = df[reviewer_cols].apply(lambda x: x.str.len().sum(), axis=1)

# -----------------------------------------------------------------------------

# We probably dont need this feature
paper_text_cols = ['title', 'keywords', 'TL;DR', 'abstract']
df_features['paper_info_length'] = df[paper_text_cols].apply(lambda x: x.str.len().sum(), axis=1)


# -----------------------------------------------------------------------------

df_features

,title,keywords,cdate,tmdate,decision_boolean,reviewer1_flag_for_ethics_review,reviewer2_flag_for_ethics_review,reviewer3_flag_for_ethics_review,reviewer4_flag_for_ethics_review,reviewer1_recommendation,...,reviewer2_tmdate,reviewer3_cdate,reviewer3_tmdate,reviewer4_cdate,reviewer4_tmdate,reviewer1_length,reviewer2_length,reviewer3_length,reviewer4_length,paper_info_length
0,Guiding Safe Exploration with Weakest Precondi...,"[reinforcement learning, safe learning, safe...",2022-09-22 14:36:24,2024-11-25 10:13:40,True,False,False,False,False,6,...,2022-10-25 00:42:31,2022-10-26 00:04:46,2022-10-26 00:04:46,2022-11-03 08:08:39,2022-12-14 03:46:36,3318.0,3517.0,1776.0,3345.0,960.0
1,An Adaptive Entropy-Regularization Framework f...,"[Multi-Agent Reinforcement Learning, Entropy ...",2022-09-22 14:33:17,2023-02-14 00:28:54,False,False,False,False,NaN,3,...,2022-10-23 18:43:07,2022-10-24 22:26:06,2022-10-24 22:26:06,NaT,NaT,5035.0,5064.0,2984.0,0.0,1259.0
2,AutoSparse: Towards Automated Sparse Training,"[sparsity, sparse training, deep learning]",2022-09-22 14:32:39,2023-02-14 00:29:06,False,False,False,False,False,5,...,2022-12-05 02:01:28,2022-10-27 05:32:06,2022-10-27 05:32:06,2022-10-27 20:14:54,2022-10-27 20:14:54,4181.0,5008.0,3123.0,2419.0,1205.0
3,Sampling is as easy as learning the score: the...,"[diffusion models, score-based generative mod...",2022-09-22 14:40:10,2023-02-14 00:27:00,True,False,False,False,False,8,...,2022-10-24 02:13:45,2022-10-25 14:35:52,2022-10-25 14:45:19,2022-10-27 22:09:32,2022-10-27 22:09:32,4834.0,1820.0,2630.0,5561.0,1418.0
4,RoCourseNet: Distributionally Robust Training ...,"[Counterfactual Explanation, Algorithmic Reco...",2022-09-22 14:38:43,2024-11-25 10:12:40,False,False,False,False,False,3,...,2022-10-24 04:04:13,2022-10-24 23:15:45,2022-10-24 23:15:45,2022-10-29 20:23:57,2022-10-29 20:23:57,3875.0,2347.0,3831.0,3200.0,1479.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,Towards Understanding Robust Memorization in A...,"[adversarial robustness, adversarial training...",2022-09-22 14:35:25,2023-02-14 00:28:19,False,False,False,False,NaN,3,...,2022-10-19 22:52:33,2022-10-25 19:10:12,2022-10-25 22:06:07,NaT,NaT,4056.0,2519.0,3055.0,0.0,1561.0
3792,Identifying Phase Transition Thresholds of Per...,[],2022-09-22 14:42:21,2023-02-14 00:26:26,False,False,False,False,False,1,...,2022-10-25 09:40:26,2022-10-25 21:04:51,2022-10-25 21:04:51,2022-10-27 21:23:15,2022-10-27 21:28:42,7448.0,5884.0,2250.0,4645.0,1335.0
3793,BEVDistill: Cross-Modal BEV Distillation for M...,"[object detection, 3d detection, BEV percept...",2022-09-22 14:30:15,2024-11-25 10:16:48,True,False,False,False,False,6,...,2022-11-18 07:50:44,2022-10-25 03:40:16,2022-11-21 18:04:58,2022-10-25 14:45:10,2022-10-25 14:45:10,2645.0,3734.0,4713.0,2808.0,1672.0
3794,Rethinking Deep Spiking Neural Networks: A Mul...,"[spiking neural network, multi-layer perceptr...",2022-09-22 14:35:28,2023-02-14 00:28:18,False,False,False,False,False,6,...,2022-10-25 22:33:12,2022-10-28 12:36:35,2022-10-28 12:36:35,2022-11-01 08:09:25,2022-11-01 08:09:25,1486.0,2535.0,2383.0,4767.0,2053.0


In [8]:
df_features.dtypes

title                                                   object
keywords                                                object
cdate                                           datetime64[ns]
tmdate                                          datetime64[ns]
decision_boolean                                          bool
reviewer1_flag_for_ethics_review                          bool
reviewer2_flag_for_ethics_review                          bool
reviewer3_flag_for_ethics_review                        object
reviewer4_flag_for_ethics_review                        object
reviewer1_recommendation                                 int64
reviewer1_confidence                                     int64
reviewer1_correctness                                    int64
reviewer1_technical_novelty_and_significance             int64
reviewer1_empirical_novelty_and_significance           float64
reviewer2_recommendation                                 int64
reviewer2_confidence                                   

In [9]:
# save
df.to_pickle(f'{dataset_csv[:-4]}-features.pkl')